# Visão Computacional - Processamento de Vídeos
Aluno: Leandro Duque Mussio

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install wfdb plotly nbformat scipy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.8 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import cv2
import numpy as np
import wfdb
from tqdm import tqdm
import re

# Caminho da pasta de vídeos e CSV
video_folder = "/content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/videos"


In [4]:
# Criar o CSV com o nome do arquivo e glicemia
video_data = [
    {"filename": "IMG_6918.MOV", "glicemia": 106},
    {"filename": "IMG_6919.MOV", "glicemia": 84},
    {"filename": "IMG_6921.MOV", "glicemia": 141},
    {"filename": "IMG_6924.MOV", "glicemia": 141},
    {"filename": "IMG_6925.MOV", "glicemia": 145},
    {"filename": "IMG_6929.MOV", "glicemia": 88},
    {"filename": "IMG_6931.MOV", "glicemia": 134},
]
df_videos = pd.DataFrame(video_data)

In [5]:
# Função para sanitizar o record_name
def sanitize_record_name(record_name):
    return re.sub(r'[^a-zA-Z0-9_-]', '_', record_name)

# Função para processar o vídeo e segmentá-lo em janelas completas de 10 segundos
def process_video_to_segments(file_path, record_id, output_folder, segment_duration=10):
    cap = cv2.VideoCapture(file_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Taxa de frames do vídeo
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Número total de frames
    segment_frames = fps * segment_duration  # Número de frames por segmento
    current_frame = 0  # Contador de frames
    segment_index = 0  # Índice do segmento

    # Calcular o número total de janelas completas de 10 segundos
    total_segments = total_frames // segment_frames

    # Barra de progresso para os segmentos
    with tqdm(total=total_segments, desc=f"Processando {record_id}", miniters=1) as segment_pbar:
        while segment_index < total_segments:
            frame_means_r = []
            frame_means_g = []
            frame_means_b = []

            # Barra de progresso para os frames do segmento
            with tqdm(total=segment_frames, desc=f"Segmento {segment_index:03d}", leave=False, miniters=30) as frame_pbar:
                for _ in range(int(segment_frames)):
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Separar os canais R, G e B
                    r_channel = frame[:, :, 2]
                    g_channel = frame[:, :, 1]
                    b_channel = frame[:, :, 0]

                    # Calcular a média dos pixels para cada canal
                    frame_means_r.append(-np.mean(r_channel))  # Inverter PPG refletido
                    frame_means_g.append(-np.mean(g_channel))  # Inverter PPG refletido
                    frame_means_b.append(-np.mean(b_channel))  # Inverter PPG refletido

                    current_frame += 1
                    frame_pbar.update(1)

            # Converter os sinais para numpy arrays
            frame_means_r = np.array(frame_means_r)
            frame_means_g = np.array(frame_means_g)
            frame_means_b = np.array(frame_means_b)

            # Combinar os sinais em um único array
            signals = np.column_stack((frame_means_r, frame_means_g, frame_means_b))

            # Nome da pasta para o segmento
            segment_folder = os.path.join(output_folder, f"{record_id}{segment_index:03d}")
            os.makedirs(segment_folder, exist_ok=True)

            # Nome do registro baseado no ID e índice do segmento
            record_name = f"{record_id}{segment_index:03d}_PPG"

            # Salvar como registro WFDB
            wfdb.wrsamp(
                record_name=record_name,
                fs=fps,  # Frequência de amostragem baseada no vídeo
                units=["a.u.", "a.u.", "a.u."],  # Unidades arbitrárias
                sig_name=["PPG_R", "PPG_G", "PPG_B"],
                p_signal=signals,
                write_dir=segment_folder
            )

            segment_index += 1
            segment_pbar.update(1)

    cap.release()


In [6]:
output_folder = "/content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/wfdb_records"

# Iterar no CSV e processar apenas os vídeos não processados
for index, row in tqdm(df_videos.iterrows(), total=len(df_videos), desc="Processando vídeos", miniters=1):
    video_path = os.path.join(video_folder, row["filename"])
    if os.path.exists(video_path):
        # Extraindo ID do vídeo
        record_id = os.path.splitext(row["filename"])[0].split('_')[1]  # Assume que o nome segue o padrão 'IMG_XXXX.MOV'

        # Verificar se já existem pastas para o vídeo no diretório de saída
        segment_folders = [f for f in os.listdir(output_folder) if f.startswith(record_id)]
        if segment_folders:
            print(f"Vídeo {record_id} já processado, ignorando...")
            continue

        # Processar o vídeo em segmentos de 10 segundos
        process_video_to_segments(video_path, record_id, output_folder, segment_duration=10)
    else:
        print(f"Arquivo não encontrado: {video_path}")

print(f"Registros WFDB segmentados e salvos em: {output_folder}")

Processando vídeos: 100%|██████████| 7/7 [00:00<00:00, 10.83it/s]

Vídeo 6918 já processado, ignorando...
Vídeo 6919 já processado, ignorando...
Vídeo 6921 já processado, ignorando...
Vídeo 6924 já processado, ignorando...
Vídeo 6925 já processado, ignorando...
Vídeo 6929 já processado, ignorando...
Vídeo 6931 já processado, ignorando...
Registros WFDB segmentados e salvos em: /content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/wfdb_records


In [7]:
import os
import pandas as pd

# Caminho do CSV de saída
csv_path = "/content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/subject-info.csv"

# Criar um DataFrame a partir dos dados de vídeo
df_video_data = pd.DataFrame(video_data)
df_video_data['record_id'] = df_video_data['filename'].apply(lambda x: os.path.splitext(x)[0].split('_')[1])  # Extrair ID do vídeo

# Lista para armazenar os dados
subject_info = []

# Percorrer todas as pastas no diretório de saída
for folder_name in os.listdir(output_folder):
    folder_path = os.path.join(output_folder, folder_name)
    if os.path.isdir(folder_path):  # Verificar se é uma pasta
        # Extrair o ID do vídeo a partir do nome da pasta (e.g., "6918000" → "6918")
        base_record_id = folder_name[:4]  # Primeiro bloco do nome
        segment_id = folder_name  # Nome completo da pasta como ID

        # Buscar o valor de glicemia correspondente
        glicemia_value = df_video_data.loc[df_video_data['record_id'] == base_record_id, 'glicemia'].values[0]

        # Adicionar ao dicionário
        subject_info.append({
            "ID": segment_id,
            "Glycaemia [mg/dL]": glicemia_value
        })

# Criar o DataFrame
df_videos = pd.DataFrame(subject_info)

# Salvar o DataFrame como CSV
df_videos.to_csv(csv_path, index=False)

print(f"CSV gerado com sucesso em: {csv_path}")

CSV gerado com sucesso em: /content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/subject-info.csv


In [8]:
import os
import re
import wfdb
import numpy as np
import pandas as pd
from tqdm import tqdm

# Base directory onde os registros estão
base_directory = "/content/drive/MyDrive/Colab Notebooks/Mestrado/Visao/wfdb_records/"

# Gerar os IDs dinamicamente a partir das pastas no diretório
record_ids = [d for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]

# Montar os caminhos completos para cada registro no formato esperado
ppg_records = [os.path.join(record_id, f"{record_id}_PPG") for record_id in record_ids]

# Variável para armazenar os dados processados
dataset = []

# Iterar sobre cada registro e processá-lo
print(f"Número total de registros: {len(ppg_records)}")
for record in tqdm(ppg_records, desc="Processando Registros"):
    # Extrair o ID do registro
    record_id = re.search(r'(\d+)', record).group(0)

    # Construir o caminho completo para o registro
    full_record_path = os.path.join(base_directory, record)

    # Carregar o sinal usando wfdb
    try:
        signals, fields = wfdb.rdsamp(full_record_path)

        # Calcular o tempo em segundos
        tempo_segundos = signals.shape[0] / fields['fs']
        t = np.linspace(0, tempo_segundos, signals.shape[0])

        # Adicionar os dados ao dataset
        dataset.append({
            'ppg_r': signals[:, 0],
            'ppg_g': signals[:, 1],
            'ppg_b': signals[:, 2],
            'time': t,
            'fs': fields['fs'],
            'sig_len': len(signals[:, 0]),
            'name': record_id
        })

    except Exception as e:
        print(f"Erro ao carregar o registro {record}: {e}")

# Criar um DataFrame com os dados processados
df = pd.DataFrame(dataset)

# Exibir as primeiras linhas do DataFrame
print(df.head())

Número total de registros: 21


Processando Registros: 100%|██████████| 21/21 [00:09<00:00,  2.27it/s]

                                               ppg_r  \
0  [-190.76369366075014, -190.72018291735262, -19...   
1  [-187.3844247589103, -187.39804160454065, -187...   
2  [-190.24881884489105, -189.7222588639106, -189...   
3  [-187.35160369467397, -187.3591275640044, -187...   
4  [-187.23699682450933, -186.99787120358874, -18...   

                                               ppg_g  \
0  [-55.76617716071521, -55.687988457100374, -55....   
1  [-53.415102683274476, -53.47135470643804, -53....   
2  [-54.98782175583263, -54.603306306198114, -54....   
3  [-53.9819054055425, -54.08115498239253, -54.15...   
4  [-53.62037382497674, -53.55031059819387, -53.5...   

                                               ppg_b  \
0  [-39.73144334663026, -39.68423086790767, -39.5...   
1  [-39.08127840074809, -39.1815379583108, -39.21...   
2  [-40.455202002548866, -40.18693863729607, -39....   
3  [-26.468694620732343, -26.557872706501872, -26...   
4  [-26.120047759913344, -26.248032390330117, 

In [9]:
# Subtrair a média de cada sinal R, G e B para remover o componente DC e controlar o deslocamento no eixo Y para plotagem
for index, row in df.iterrows():
    df.at[index, 'ppg_r'] = row['ppg_r'] - np.mean(row['ppg_r'])
    df.at[index, 'ppg_g'] = row['ppg_g'] - np.mean(row['ppg_g']) -4
    df.at[index, 'ppg_b'] = row['ppg_b'] - np.mean(row['ppg_b']) -8

print("Componente DC removido!")
print(df.head())

Componente DC removido!
                                               ppg_r  \
0  [-1.2959714477879345, -1.2524607043904155, -1....   
1  [1.3583655242951806, 1.3447486786648426, 1.341...   
2  [-2.102925538069087, -1.57636555708865, -0.879...   
3  [1.1677389570102434, 1.160215087679802, 1.1168...   
4  [1.704371481038521, 1.9434971019591103, 2.0839...   

                                               ppg_g  \
0  [-4.880835352379329, -4.802646648764494, -4.67...   
1  [-3.093813880770014, -3.15006590393358, -3.197...   
2  [-5.140915486309623, -4.75640003667511, -4.324...   
3  [-3.0072847646217937, -3.1065343414718285, -3....   
4  [-2.4108593250367107, -2.34079609825384, -2.34...   

                                               ppg_b  \
0  [-8.112825813896613, -8.065613335174028, -7.95...   
1  [-7.11771665517432, -7.217976212737028, -7.249...   
2  [-8.49767714755788, -8.229413782305087, -7.916...   
3  [-7.795634395477268, -7.884812481246797, -7.94...   
4  [-6.839689598866187

In [10]:
print(df.shape)
print(df.head)

(21, 7)
<bound method NDFrame.head of                                                 ppg_r  \
0   [-1.2959714477879345, -1.2524607043904155, -1....   
1   [1.3583655242951806, 1.3447486786648426, 1.341...   
2   [-2.102925538069087, -1.57636555708865, -0.879...   
3   [1.1677389570102434, 1.160215087679802, 1.1168...   
4   [1.704371481038521, 1.9434971019591103, 2.0839...   
5   [-2.109909187791885, -2.583506838991582, -2.98...   
6   [-1.3231844660940624, -1.3713312071098187, -1....   
7   [-0.6356026048106003, -0.795814950224127, -0.9...   
8   [-0.5476532892974433, -0.7579630497915844, -0....   
9   [0.3003247209478843, 0.34858710511144864, 0.32...   
10  [-2.655238816274334, -2.9508459799377533, -3.3...   
11  [-2.7974840181402953, -3.053776278816315, -3.1...   
12  [0.6080093802395083, 1.0468737836172863, 1.130...   
13  [-3.0712225088273613, -3.226919874333504, -3.2...   
14  [3.3737756736722417, 3.6015020577667087, 3.626...   
15  [-0.37629594151434276, -0.41718267773222806, -

In [11]:
import plotly.graph_objects as go

indices = [12]
fig = go.Figure()
for i, index in enumerate(indices):
    print(index)
    fig.add_trace(go.Scatter(x=df['time'][index], y=df['ppg_r'][index], mode='lines', name=df['name'][index], line=dict(color='red') ))
    fig.add_trace(go.Scatter(x=df['time'][index], y=df['ppg_g'][index], mode='lines', name=df['name'][index], line=dict(color='green') ))
    fig.add_trace(go.Scatter(x=df['time'][index], y=df['ppg_b'][index], mode='lines', name=df['name'][index], line=dict(color='blue') ))

fig.update_layout(title='PPG Signals')
fig.show()


12
